In [223]:
import pandas as pd
import numpy as np
import re
import openpyxl

from metar_taf_parser.parser.parser import MetarParser

Metars de ejemplo, en la practica se puede recorrer un archivo csv y obtener la lista de metars a usar

In [224]:
metares = ['KTTN 051853Z 04011KT 9999 VCTS SN FZFG BKN003 OVC010 M02/M02 A3006',
           'MMAA 021442Z 35003KT 6SM SKC 29/19 A2987 RMK SLP116 52025 900 HZY ISOL CI',
           'MMGL 021444Z 00000KT 7SM SCT300 21/01 A3016 RMK SLP099 52014 904 8/002 HZY',
           'MMMX 021515Z 19003KT 8SM SKC 23/02 A3028 NOSIG RMK HZY RMK 8/100']

In [225]:
df = pd.DataFrame(columns=['ICAO', 'day', 'time', 'wind_dir', 'wind_speed', 'wind_gust', 'visibility', 'weather', 'cloudsheight1', 'cloudsquantity1', 
                           'cloudsheight2', 'cloudsquantity2', 'cloudsheight3', 'cloudsquantity3',
                           'temperature', 'dewpoint', 'altimeter', 'remark'])

In [226]:
df_metars = pd.DataFrame() 
for metar in metares:
    parse = MetarParser().parse(metar)
    df_metars['ICAO'] = [parse.station]
    df_metars['day'] = [parse.day]
    df_metars['time'] = [parse.time]
    df_metars['wind_dir'] = [parse.wind.degrees]
    df_metars['wind_speed'] = [parse.wind.speed]
    df_metars['wind_gust'] = [parse.wind.gust if parse.wind.gust else np.nan]
    df_metars['visibility'] = [parse.visibility.distance.replace('SM', '')]
    df_metars['weather'] = [parse.weather_conditions[0].descriptive.name if parse.weather_conditions else np.nan]
    df_metars['cloudsheight1'] = [parse.clouds[0].height if len(parse.clouds) >= 1 else np.nan]
    df_metars['cloudsquantity1'] = [parse.clouds[0].quantity.name if parse.clouds[0].quantity.name else np.nan]
    df_metars['cloudsheight2'] = [parse.clouds[1].height if len(parse.clouds) >= 2 else np.nan]
    df_metars['cloudsquantity2'] = [parse.clouds[1].quantity.name if len(parse.clouds) >= 2 else np.nan]
    df_metars['cloudsheight3'] = [parse.clouds[2].height if len(parse.clouds) >= 3 else np.nan]
    df_metars['cloudsquantity3'] = [parse.clouds[2].quantity.name if len(parse.clouds) >= 3 else np.nan]
    df_metars['temperature'] = [parse.temperature]
    df_metars['dewpoint'] = [parse.dew_point]
    df_metars['altimeter'] = [parse.altimeter]
    RE = r'^8/.{3}$'
    remarks = parse.remarks
    rmk = [s for s in remarks if re.match(RE, s)]
    df_metars['remark'] = [rmk[0] if rmk else np.nan]
    df = pd.concat([df, df_metars], ignore_index=True)

df.to_excel('metars.xlsx', index=False)

In [227]:
df

,ICAO,day,time,wind_dir,wind_speed,wind_gust,visibility,weather,cloudsheight1,cloudsquantity1,cloudsheight2,cloudsquantity2,cloudsheight3,cloudsquantity3,temperature,dewpoint,altimeter,remark
0,KTTN,5,18:53:00,40,11,NaN,> 10km,THUNDERSTORM,300,BKN,1000,OVC,NaN,NaN,-2,-2,1017,NaN
1,MMAA,2,14:42:00,350,3,NaN,6,NaN,None,SKC,NaN,NaN,NaN,NaN,29,19,1011,NaN
2,MMGL,2,14:44:00,0,0,NaN,7,NaN,30000,SCT,NaN,NaN,NaN,NaN,21,1,1021,8/002
3,MMMX,2,15:15:00,190,3,NaN,8,NaN,None,SKC,NaN,NaN,NaN,NaN,23,2,1025,8/100
